# Using `pybind11`

The package `pybind11` is provides an elegant way to wrap C++ code for Python, including automatic conversions for `numpy` arrays and the C++ `Eigen` linear algebra library. Used with the `cppimport` package, this provides a very nice work flow for integrating C++ and Python:

- Edit C++ code
- Run Python code

```bash
! pip install pybind11
! pip install cppimport
```

Clone the Eigen library if necessary - no installation is required as Eigen is a header only library.

```bash
! git clone https://github.com/RLovelett/eigen.git
```

## Resources

- [`pybind11`](http://pybind11.readthedocs.io/en/latest/)
- [`cppimport`](https://github.com/tbenthompson/cppimport)
- [`Eigen`](http://eigen.tuxfamily.org)

## A first example of using `pybind11`

Create a new subdirectory - e.g. `example1` and create the following 5 files in it:

- `funcs.hpp`
- `funcs.cpp`
- `wrap.cpp`
- `setup.py`
- `test_funcs.py`

First write the C++ header and implementation files

In [2]:
%mkdir example1
%cd example1

/home/jovyan/work/sta-663-2018/notebooks/example1


In [42]:
%%file funcs.hpp

int add(int i, int j);

Writing funcs.hpp


In [38]:
%%file funcs.cpp

int add(int i, int j) {
    return i + j;
};

Writing funcs.cpp


Next write the C++ wrapper code using `pybind11` in `wrap.cpp`. The arguments `"i"_a=1, "j"_a=2` in the exported function definition tells `pybind11` to generate variables named `i` with default value 1 and `j` with default value 2 for the `add` function.

In [47]:
%%file wrap1.cpp
#include <pybind11/pybind11.h>
#include "funcs.hpp"

namespace py = pybind11;

using namespace pybind11::literals;

PYBIND11_MODULE(wrap1, m) {
    m.doc() = "pybind11 example plugin"; 
    m.def("add", &add, "A function which adds two numbers",
          "i"_a=1, "j"_a=2);
}

Overwriting wrap1.cpp


Finally, write the `setup.py` file to compile the extension module. This is mostly boilerplate.

In [48]:
%%file setup.py
import os, sys

from distutils.core import setup, Extension
from distutils import sysconfig

cpp_args = ['-std=c++11'] 

ext_modules = [
    Extension(
    'wrap1',
        ['funcs.cpp', 'wrap1.cpp'],
        include_dirs=['pybind11/include'],
    language='c++',
    extra_compile_args = cpp_args,
    ),
]

setup(
    name='wrap1',
    version='0.0.1',
    author='Cliburn Chan',
    author_email='cliburn.chan@duke.edu',
    description='Example',
    ext_modules=ext_modules,
)

Overwriting setup.py


Now build the extension module in the subdirectory with these files

In [49]:
%%bash

python setup.py build_ext -i

running build_ext
building 'wrap1' extension
gcc -pthread -B /opt/conda/compiler_compat -Wl,--sysroot=/ -Wsign-compare -DNDEBUG -g -fwrapv -O3 -Wall -Wstrict-prototypes -fPIC -Ipybind11/include -I/opt/conda/include/python3.6m -c funcs.cpp -o build/temp.linux-x86_64-3.6/funcs.o -std=c++11
gcc -pthread -B /opt/conda/compiler_compat -Wl,--sysroot=/ -Wsign-compare -DNDEBUG -g -fwrapv -O3 -Wall -Wstrict-prototypes -fPIC -Ipybind11/include -I/opt/conda/include/python3.6m -c wrap1.cpp -o build/temp.linux-x86_64-3.6/wrap1.o -std=c++11
g++ -pthread -shared -B /opt/conda/compiler_compat -L/opt/conda/lib -Wl,-rpath=/opt/conda/lib -Wl,--no-as-needed -Wl,--sysroot=/ build/temp.linux-x86_64-3.6/funcs.o build/temp.linux-x86_64-3.6/wrap1.o -L/opt/conda/lib -lpython3.6m -o /home/jovyan/work/sta-663-2018/notebooks/example6/wrap1.cpython-36m-x86_64-linux-gnu.so


cc1plus: warning: command line option ‘-Wstrict-prototypes’ is valid for C/ObjC but not for C++
cc1plus: warning: command line option ‘-Wstrict-prototypes’ is valid for C/ObjC but not for C++


And if you are successful, you should now see a new ```funcs.so``` extension module. We can write a `test_funcs.py` file test the extension module:

In [50]:
%%file test_funcs.py

import wrap1

def test_add():
    print(wrap1.add(3, 4))
    assert(wrap1.add(3, 4) == 7)

if __name__ == '__main__':
    test_add()

Writing test_funcs.py


And finally, running the test should not generate any error messages:

In [51]:
%%bash

python test_funcs.py

7


In [52]:
%cd ..

/home/jovyan/work/sta-663-2018/notebooks


## Using `cppimport`

In the development stage, it can be distracting to have to repeatedly rebuild the extension module by running

```bash
python setup.py clean
python setup.py build_ext -i
```

every single time you modify the C++ code. The `cppimport` package does this for you. 

Create a new sub-directory `exaample2` and copy the files `func.hpp`, `funcs.cpp` and `wrap.cpp` from `example1` over.
For the previous example, we just need to add some annotation (between `<% and %>` delimiters) to the top of the `wrap.cpp` file

In [53]:
%mkdir example2
%cp example1/funcs.* example2/
%cd example2

mkdir: cannot create directory ‘example2’: File exists
/home/jovyan/work/sta-663-2018/notebooks/example2


In [54]:
%%file wrap2.cpp
<%
cfg['compiler_args'] = ['-std=c++11'] 
cfg['sources'] = ['funcs.cpp']
setup_pybind11(cfg)
%>

#include "funcs.hpp"
#include <pybind11/pybind11.h>

namespace py = pybind11;

PYBIND11_MODULE(wrap2, m) {
    m.doc() = "pybind11 example plugin"; 
    m.def("add", &add, "A function which adds two numbers");
}

Overwriting wrap2.cpp


In [55]:
%%file test_funcs.py

import cppimport
funcs = cppimport.imp("wrap2")

def test_add():
    assert(funcs.add(3, 4) == 7)

if __name__ == '__main__':
    print(funcs.add(3,4))
    test_add()

Overwriting test_funcs.py


In [56]:
%%bash

python test_funcs.py 

7


cc1plus: warning: command line option ‘-Wstrict-prototypes’ is valid for C/ObjC but not for C++
cc1plus: warning: command line option ‘-Wstrict-prototypes’ is valid for C/ObjC but not for C++


### Use of `cppimport`

Note that `cppimport.imp` is only called once. Once it is called, the shared library is created and can be sued

In [58]:
! ls *so

wrap2.cpython-36m-x86_64-linux-gnu.so


That is, you can import wrap2 and call from notebook

In [59]:
import wrap2

wrap2.add(3, 4)

7

without any need to manually build the extension module. Any updates will be detected by `cppimport` and it will automatically trigger a re-build.

In [60]:
%cd ..

/home/jovyan/work/sta-663-2018/notebooks


## Vectorizing functions for use with `numpy` arrays

Example showing how to vectorize a `square` function. Note that from here on, we don't bother to use separate header and implementation files for these code snippets, and just write them together with the wrapping code in a `code.cpp` file. This means that with `cppimport`, there are only two files that we actually code for, a C++ `code.cpp` file and a python test file.

In [61]:
%mkdir example3
%cd example3

mkdir: cannot create directory ‘example3’: File exists
/home/jovyan/work/sta-663-2018/notebooks/example3


In [66]:
%%file wrap3.cpp
<%
cfg['compiler_args'] = ['-std=c++11'] 
setup_pybind11(cfg)
%>

#include <pybind11/pybind11.h>
#include <pybind11/numpy.h>

namespace py = pybind11;
double square(double x) {
    return x * x;
}

PYBIND11_MODULE(wrap3, m) {
    m.doc() = "pybind11 example plugin";
    m.def("square", py::vectorize(square), "A vectroized square function.");
}

Overwriting wrap3.cpp


In [67]:
import cppimport

wrap3 = cppimport.imp("wrap3")
wrap3.square([1,2,3])

array([ 1.,  4.,  9.])

Once the shared libary is built, you can use it as a regular Python module.

In [68]:
! ls

wrap3.cpp  wrap3.cpython-36m-x86_64-linux-gnu.so


In [69]:
import wrap3

wrap3.square([2,4,6])

array([  4.,  16.,  36.])

In [70]:
%cd ..

/home/jovyan/work/sta-663-2018/notebooks


## Using `numpy` arrays as function arguments and return values

Example showing how to pass `numpy` arrays in and out of functions. These `numpy` array arguments can either be generic `py:array` or typed `py:array_t<double>`. The properties of the `numpy` array can be obtained by calling its `request` method. This returns a `struct` of the following form:

```c++
struct buffer_info {
    void *ptr;
    size_t itemsize;
    std::string format;
    int ndim;
    std::vector<size_t> shape;
    std::vector<size_t> strides;
};
```

Here is C++ code for two functions - the function `twice` shows how to change a passed in `numpy` array in-place using pointers; the function `sum` shows how to sum the elements of a `numpy` array. By taking advantage of the information in `buffer_info`, the code will work for arbitrary `n-d` arrays.

In [71]:
%mkdir example4
%cd example4

mkdir: cannot create directory ‘example4’: File exists
/home/jovyan/work/sta-663-2018/notebooks/example4


In [72]:
%%file wrap4.cpp
<%
cfg['compiler_args'] = ['-std=c++11']
setup_pybind11(cfg)
%>

#include <pybind11/pybind11.h>
#include <pybind11/numpy.h>

namespace py = pybind11;

// Passing in an array of doubles
void twice(py::array_t<double> xs) {
    py::buffer_info info = xs.request();
    auto ptr = static_cast<double *>(info.ptr);

    int n = 1;
    for (auto r: info.shape) {
      n *= r;
    }

    for (int i = 0; i <n; i++) {
        *ptr++ *= 2;
    }
}

// Passing in a generic array
double sum(py::array xs) {
    py::buffer_info info = xs.request();
    auto ptr = static_cast<double *>(info.ptr);

    int n = 1;
    for (auto r: info.shape) {
      n *= r;
    }

    double s = 0.0;
    for (int i = 0; i <n; i++) {
        s += *ptr++;
    }

    return s;
}

PYBIND11_MODULE(wrap4, m) {
    m.doc() = "auto-compiled c++ extension";
    m.def("sum", &sum);
    m.def("twice", &twice);
}

Overwriting wrap4.cpp


In [73]:
%%file test_code.py
import cppimport
import numpy as np

code = cppimport.imp("wrap4")

if __name__ == '__main__':
    xs = np.arange(12).reshape(3,4).astype('float')
    print(xs)
    print("np :", xs.sum())
    print("cpp:", code.sum(xs))

    print()
    code.twice(xs)
    print(xs)

Overwriting test_code.py


In [74]:
%%bash

python test_code.py

[[  0.   1.   2.   3.]
 [  4.   5.   6.   7.]
 [  8.   9.  10.  11.]]
np : 66.0
cpp: 66.0

[[  0.   2.   4.   6.]
 [  8.  10.  12.  14.]
 [ 16.  18.  20.  22.]]


cc1plus: warning: command line option ‘-Wstrict-prototypes’ is valid for C/ObjC but not for C++


In [75]:
%cd ..

/home/jovyan/work/sta-663-2018/notebooks


## More on working with `numpy` arrays

This example shows how to use array access for `numpy` arrays within the C++ function. It is taken from the `pybind11` documentation, but fixes a small bug in the official version. As noted in the documentation, the function would be more easily coded using `py::vectorize`.

In [76]:
%mkdir example5
%cd example5

mkdir: cannot create directory ‘example5’: File exists
/home/jovyan/work/sta-663-2018/notebooks/example5


In [77]:
%%file wrap5.cpp
<%
cfg['compiler_args'] = ['-std=c++11']
setup_pybind11(cfg)
%>

#include <pybind11/pybind11.h>
#include <pybind11/numpy.h>

namespace py = pybind11;

py::array_t<double> add_arrays(py::array_t<double> input1, py::array_t<double> input2) {
    auto buf1 = input1.request(), buf2 = input2.request();

    if (buf1.ndim != 1 || buf2.ndim != 1)
        throw std::runtime_error("Number of dimensions must be one");

    if (buf1.shape[0] != buf2.shape[0])
        throw std::runtime_error("Input shapes must match");

    auto result = py::array(py::buffer_info(
        nullptr,            /* Pointer to data (nullptr -> ask NumPy to allocate!) */
        sizeof(double),     /* Size of one item */
        py::format_descriptor<double>::value, /* Buffer format */
        buf1.ndim,          /* How many dimensions? */
        { buf1.shape[0] },  /* Number of elements for each dimension */
        { sizeof(double) }  /* Strides for each dimension */
    ));

    auto buf3 = result.request();

    double *ptr1 = (double *) buf1.ptr,
           *ptr2 = (double *) buf2.ptr,
           *ptr3 = (double *) buf3.ptr;

    for (size_t idx = 0; idx < buf1.shape[0]; idx++)
        ptr3[idx] = ptr1[idx] + ptr2[idx];

    return result;
}

PYBIND11_MODULE(wrap5, m) {
    m.def("add_arrays", &add_arrays, "Add two NumPy arrays");
}

Overwriting wrap5.cpp


In [78]:
import cppimport
import numpy as np

code = cppimport.imp("wrap5")

xs = np.arange(12)
print(xs)

print(code.add_arrays(xs, xs))

[ 0  1  2  3  4  5  6  7  8  9 10 11]
[  0.   2.   4.   6.   8.  10.  12.  14.  16.  18.  20.  22.]


In [79]:
%cd ..

/home/jovyan/work/sta-663-2018/notebooks


## Using the C++ `eigen` library to calculate matrix inverse and determinant

Example showing how `Eigen` vectors and matrices can be passed in and out of C++ functions. Note that `Eigen` arrays are automatically converted to/from `numpy` arrays simply by including the `pybind/eigen.h` header. Because of this, it is probably simplest in most cases to work with `Eigen` vectors and matrices rather than `py::buffer` or `py::array` where `py::vectorize` is insufficient.

In [80]:
%mkdir example6
%cd example6

mkdir: cannot create directory ‘example6’: File exists
/home/jovyan/work/sta-663-2018/notebooks/example6


In [85]:
%%file wrap6.cpp
<%
cfg['compiler_args'] = ['-std=c++11']
cfg['include_dirs'] = ['../eigen3']
setup_pybind11(cfg)
%>

#include <pybind11/pybind11.h>
#include <pybind11/eigen.h>

#include <Eigen/LU>

namespace py = pybind11;

// convenient matrix indexing comes for free
double get(Eigen::MatrixXd xs, int i, int j) {
    return xs(i, j);
}

// takes numpy array as input and returns double
double det(Eigen::MatrixXd xs) {
    return xs.determinant();
}

// takes numpy array as input and returns another numpy array
Eigen::MatrixXd inv(Eigen::MatrixXd xs) {
    return xs.inverse();
}

PYBIND11_MODULE(wrap6, m) {
    m.doc() = "auto-compiled c++ extension";
    m.def("inv", &inv);
    m.def("det", &det);
}

Overwriting wrap6.cpp


In [86]:
import cppimport
import numpy as np

code = cppimport.imp("wrap6")

A = np.array([[1,2,1],
              [2,1,0],
              [-1,1,2]])

print(A)
print(code.det(A))
print(code.inv(A))

[[ 1  2  1]
 [ 2  1  0]
 [-1  1  2]]
-3.0
[[-0.66666667  1.          0.33333333]
 [ 1.33333333 -1.         -0.66666667]
 [-1.          1.          1.        ]]


In [87]:
%cd ..

/home/jovyan/work/sta-663-2018/notebooks


## Using `pybind11` with `openmp`

In [88]:
%mkdir example7
%cd example7

/home/jovyan/work/sta-663-2018/notebooks/example7


Here is an example of using OpenMP to integrate the value of $\pi$ written using `pybind11`.

In [89]:
%%file wrap7.cpp
/*
<%
cfg['compiler_args'] = ['-std=c++11', '-fopenmp']
cfg['linker_args'] = ['-lgomp']
setup_pybind11(cfg)
%>
*/
#include <cmath>
#include <omp.h>
#include <pybind11/pybind11.h>
#include <pybind11/numpy.h>

namespace py = pybind11;

// Passing in an array of doubles
void twice(py::array_t<double> xs) {
    py::gil_scoped_acquire acquire;
    
    py::buffer_info info = xs.request();
    auto ptr = static_cast<double *>(info.ptr);

    int n = 1;
    for (auto r: info.shape) {
      n *= r;
    }

    #pragma omp parallel for
    for (int i = 0; i <n; i++) {
        *ptr++ *= 2;
    }
}  

PYBIND11_MODULE(wrap7, m) {
  m.doc() = "auto-compiled c++ extension";
  m.def("twice", [](py::array_t<double> xs) {
      /* Release GIL before calling into C++ code */      
      py::gil_scoped_release release;
      return twice(xs);
    });
}

Writing wrap7.cpp


In [90]:
import cppimport
import numpy as np

code = cppimport.imp("wrap7")
xs = np.arange(10).astype('double')
code.twice(xs)
xs

array([  0.,   2.,   4.,   6.,   8.,  10.,  12.,  14.,  16.,   9.])

In [91]:
%cd ..

/home/jovyan/work/sta-663-2018/notebooks
